<a href="https://colab.research.google.com/github/justinballas/WorkAutomation/blob/main/BICFR_selector_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 

In [ ]:
df = pd.read_csv('20210228_GW_PPVUL.csv', index_col='Name')

#if you need to clean the fund names
replace_list = ["US Insurance ","US Fund "]

for term in replace_list:
  df.iloc[1] = df.iloc[1].str.replace(term, "")


#enter columns to use for ranking with their associated weights
rank_and_weight = {
    'Morningstar \nRating \nOverall': 1,
    'Morningstar \nRating \n3 Yr': 1,
    'Morningstar \nRating \n5 Yr': 1,
    'Morningstar \nRating \n10 Yr': 1,
    'Total Ret \n% Rank Cat \n1 Yr (Mo-End)': 1,
    'Total Ret \n% Rank Cat \n3 Yr (Mo-End)': 1,
    'Total Ret \n% Rank Cat \n5 Yr (Mo-End)': 1,
    'Total Ret \n% Rank Cat \n10 Yr (Mo-End)': 1,
    'Sharpe Ratio \n3 Yr (Mo-End) \nRisk \nCurrency': 1,
    'Manager \nTenure \n(Average)': 1
}

#Return all selected metrics [y/n]?
#n for only selected fund and asset class
return_ans = 'n'

#name of excel file when exported
file_name = 'Best_in_class_selections'

columns = list(rank_and_weight.keys())
category_col = df['Morningstar Category']

#selecting columns to scale
df2 = df.loc[:, columns]
x=df2.values

#scaling selected colums
min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1)) 
new_df = min_max_scaler.fit_transform(x)
scaled_df = pd.DataFrame(new_df, index=df.index, columns=list(rank_and_weight.keys()))

#applying weight and reversing weight of "lower is better" columns
for k, v in rank_and_weight.items():
  if 'Rank Cat' in k:
    scaled_df[k] = scaled_df[k].apply(lambda x: (1-x)*v)
  else:
    scaled_df[k] = scaled_df[k].apply(lambda x: x*v)

#calculating overall score
scaled_df['Overall Score'] = scaled_df.sum(axis=1)

#ranking overall score
scaled_df['Overall Rank'] = scaled_df['Overall Score'].rank(ascending=True)

#adding back morningstar categories
scaled_df['Morningstar Category'] = category_col

#creating list of asset classes to iterate over
asset_classes = list(category_col.unique())

#getting highest ranked fund per asset class
max_asset_classes = {}
for asset_class in asset_classes:
  class_df = scaled_df[scaled_df['Morningstar Category']==asset_class]
  bicf = class_df['Overall Rank'].idxmax()
  max_asset_classes[asset_class] = bicf

#creating dataframe and naming excel file
if return_ans =='y':
  export_df = df.loc[list(max_asset_classes.values())]
else:
  export_df = pd.DataFrame.from_dict(max_asset_classes, orient='index')

#exporting to excel
#export_df.to_excel(file_name + '.xlsx')

export_df

In [ ]:
#df.loc[list(max_asset_classes.values())]